In [76]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [77]:
!python -V

Python 3.9.6


In [78]:
import os
import pickle
import pandas as pd

In [79]:
year = 2023
month = 3
filename = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet"
output_file = 'output.parquet'
print(filename)
df = read_data(filename)

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet


In [80]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/girish_balwani/girish/girish-mlops/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/girish_balwani/girish/girish-mlops/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [81]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [82]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
y_pred


array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [83]:
import numpy as np
std_dev = np.std(y_pred)
print(f"The standard deviation of the predicted duration is: {std_dev}")

The standard deviation of the predicted duration is: 6.247488852238703


In [84]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [85]:
df_result = df[['ride_id']].copy()  # Include only the `ride_id` column initially
df_result['prediction'] = y_pred   # Add predictions as a new column

print (df_result)

                 ride_id  prediction
0              2023/03_0   16.245906
1              2023/03_1   26.134796
2              2023/03_2   11.884264
3              2023/03_3   11.997720
4              2023/03_4   10.234486
...                  ...         ...
3403761  2023/03_3403761   11.952635
3403762  2023/03_3403762   20.049958
3403763  2023/03_3403763   11.595336
3403764  2023/03_3403764   13.113178
3403765  2023/03_3403765   12.899992

[3316216 rows x 2 columns]


In [86]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [87]:
output_size = round(os.path.getsize(output_file) / (1024 * 1024), 2)
print (f"new parquet file size is {output_size} MB")


new parquet file size is 65.46 MB


In [88]:
dg = pd.read_parquet(output_file)
dg


,ride_id,prediction
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3316211,2023/03_3403761,11.952635
3316212,2023/03_3403762,20.049958
3316213,2023/03_3403763,11.595336
3316214,2023/03_3403764,13.113178


In [89]:
mean_predicted_duration = round(np.mean(y_pred), 2)  # Calculate mean
print(f"Mean Predicted Duration: {mean_predicted_duration}")

Mean Predicted Duration: 14.2
